In [142]:
import numpy as np
import sys,os
from ase.io import read,write
from ase.visualize import view
import ase
import cPickle as pck
import pandas as pd
import spglib as spg
from tqdm import tqdm_notebook

In [74]:
sys.path.insert(0,'../make_input/')

In [80]:
from qe_input import makeQEInput
from custom_frame import get_ibrav0_frame
from utils import get_symprec

# import info and structures

In [71]:
fileNames = {}
infoPath = '../info/'
structurePath = '../structures/'
fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [72]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

In [77]:
sg,it = 79,3
cc = crystals[sg][it]
dd,_,_ = get_ibrav0_frame(cc,sg)


# get filenames

In [38]:
import fnmatch
import os

path = '../test_run/check_ibrav0_sym/'
matches = []
for root, dirnames, filenames in os.walk(path):
    for dirname in fnmatch.filter(dirnames, 'sg_80-*'):
        filenames = os.listdir(os.path.join(root,dirname))
        for filename in fnmatch.filter(filenames, 'qe.out'):
            matches.append(os.path.abspath(os.path.join(root,dirname, filename)))

In [40]:
matches

['/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_4-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_1-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_2-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_0-0/qe.out',
 '/local/git/run_qe/test_run/check_ibrav0_sym/sg_80-f_3-0/qe.out']

In [32]:
os.listdir('../test_run/check_ibrav0_sym/sg_80-f_0-0')

['qe.in', 'qe.err', 'qe.out']

In [43]:
def get_filenames(path,fn_pattern='qe.out',dir_pattern=None):
    import fnmatch
    import os
    matches = []
    if dir_pattern is None:
        # Find the paths of all the filename with fn_pattern within path
        for root, dirnames, filenames in os.walk(path):
            for filename in fnmatch.filter(filenames, fn_pattern):
                matches.append(os.path.join(root, filename))
        return matches
    else:
        # Find the paths with dir_pattern and all the filename with fn_pattern within dir_pattern (no recursion)
        for root, dirnames, filenames in os.walk(path):
            for dirname in fnmatch.filter(dirnames, dir_pattern):
                filenames = os.listdir(os.path.join(root,dirname))
                for filename in fnmatch.filter(filenames, fn_pattern):
                    matches.append(os.path.abspath(os.path.join(root,dirname, filename)))


In [44]:
path = '../test_run/check_ibrav0_sym/'
fns = get_filenames(path,fn_pattern='qe.out',dir_pattern=None)

# parse files

## some test

In [59]:
import re
import sys
import glob

str_patterns = ['Sym. Ops.', '!','tau']
str_patterns = ['Sym. Ops.']
matches = {key:[] for key in str_patterns}

for fn in fns:
    for line in open(fn, 'r'):
        for str_pattern in str_patterns:
            if re.search(str_pattern, line):
                matches[str_pattern].append(line.replace('\n',''))
print matches['Sym. Ops.']

['../test_run/check_ibrav0_sym/sg_79-f_3-0/qe.out']
['         1           Si  tau(   1) = (  -0.2081053   0.3878910   0.5381366  )', '         2           Si  tau(   2) = (   0.8040956   0.2080994   0.0000000  )', '         3           Si  tau(   3) = (   0.2080994   0.3878851   0.0000000  )', '         4           Si  tau(   4) = (   0.3878910   0.2081053   0.5381366  )', '         1           Si  tau(   1) = (  0.8254172  0.3254122  0.1508294  )', '         2           Si  tau(   2) = (  0.1745828  0.6745878  0.8491706  )', '         3           Si  tau(   3) = (  0.3254122  0.1745828  0.4999951  )', '         4           Si  tau(   4) = (  0.6745878  0.8254172  0.5000049  )']


In [90]:
def get_patterns(fn,str_patterns):
    matches = {key:[] for key in str_patterns}
    
    for line in open(fn, 'r'):
        for str_pattern in str_patterns:
            if re.search(str_pattern, line):
                matches[str_pattern].append(line.replace('\n',''))
    return matches

In [64]:
cc = read(fns[0])
view(cc)

In [129]:
fn = fns[0]
ms = re.search("sg_(.*)-f_",fn)
sg = int(ms.group(1))
print sg

79


In [131]:
def fn2sg(fn,pattern="sg_(.*)-f_"):
    ms = re.search(pattern,fn)
    sg = int(ms.group(1))
    return sg
fn2sg(fn)

79

In [83]:
re.findall("[-\d]+", "he33llo 42 I\'m a 32.3 string -30 1 -2")

['33', '42', '32', '3', '-30', '1', '-2']

In [84]:
s = """4.5 abc -4.5 abc - 4.5 abc + .1e10 abc . abc 1.01e-2 abc 
       1.01e-.2 abc 123 abc .123"""
print re.findall(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?", s)


['4.5', '-4.5', '- 4.5', '+ .1e10', ' 1.01e-2', '       1.01', '-.2', ' 123', ' .123']


[4.5, -4.5, 4.5, 1000000000.0, 0.0101, 123.0, 0.123]


In [139]:
def extract_floats(s):
    l = []
    for t in s.split():
        try:
            l.append(float(t))
        except ValueError:
            pass
    return l

In [89]:
%%timeit -r 3 -n 20
extract_float(s)

20 loops, best of 3: 16.3 µs per loop


## get number of detected symmetries

In [200]:
path = '../test_run/check_ibrav0_sym/'
fns_out = get_filenames(path,fn_pattern='qe.out',dir_pattern=None)
fns_in = get_filenames(path,fn_pattern='qe.in',dir_pattern=None)

In [201]:
fn = fns_in[14]
print fn

../test_run/check_ibrav0_sym/sg_70-f_4-0/qe.in


In [202]:
cc = read(fn)
sg = fn2sg(fn)
symprec = get_symprec(cc,sg)
syms = spg.get_symmetry(cc,symprec=symprec)
Nsym_s = syms['rotations'].shape[0]
print Nsym_s

8


In [164]:
view(cc)

In [154]:
from raw_info import missClassificationCorrection
missClassificationCorrection[70]

KeyError: 70

In [197]:
sg,it = 43,1
cc = crystals[sg][it]
dd,_,_ = get_ibrav0_frame(cc,sg)
view(dd)
print dd.get_cell()

[[ 0.          7.186617    1.52812703]
 [ 1.62630392  0.          1.52812703]
 [ 1.62630392  7.186617    0.        ]]


In [198]:
fn

'../test_run/check_ibrav0_sym/sg_43-f_1-0/qe.out'

In [203]:

# print get_patterns(fn,str_patterns=['Sym. Ops.']).values()
# s = get_patterns(fn,str_patterns=['Sym. Ops.']).values()[0][0]
# Nsym_f = extract_floats(s)[0]
fn = '../test_run/check_ibrav0_sym/sg_43-f_1-0/qe.in'
cc = read(fn)
sg = fn2sg(fn)
# view(cc)
print cc.get_cell()
print sg
symprec = get_symprec(cc,sg)
print symprec
# syms = spg.get_symmetry(cc,symprec=symprec)

# Nsym_s = syms['rotations'].shape[0]

# print Nsym_s == Nsym_f
# print s


[[ 0.          7.186617    1.52812703]
 [ 1.62630392  0.          1.52812703]
 [ 1.62630392  7.186617    0.        ]]
43
1e-05


In [236]:
fn = fns_out[5]
print fn
ss = get_patterns(fn,str_patterns=['Sym. Ops.','warning: symmetry operation #','No symmetry found',
                                          'additional sym.ops. were found but ignored'])
Nsym_f = 0
print ss['additional sym.ops. were found but ignored']
if ss['additional sym.ops. were found but ignored']:
    s = ss['additional sym.ops. were found but ignored'][0]
    Nsym_f += extract_floats(s)[0]
print Nsym_f
# if ss['warning: symmetry operation #']:
#     nums = []
#     for s in ss['warning: symmetry operation #']:
#         nums.append(extract_floats(s)[0])
    
#     nums = np.array(nums).max()
#     Nsym_f = nums.max()
# else:
#     print ss['Sym. Ops.']
#     s = ss['Sym. Ops.'][0][0]
#     Nsym_f = extract_floats(s)[0]

# cc = read(fn)
# sg = fn2sg(fn)
# symprec = get_symprec(cc,sg)
# syms = spg.get_symmetry(cc,symprec=symprec)
# Nsym_s = syms['rotations'].shape[0]

# print Nsym_s == Nsym_f
# print s


../test_run/check_ibrav0_sym/sg_40-f_0-0/qe.out
['          (note:  4 additional sym.ops. were found but ignored']
4.0


In [273]:
path = '../test_run/check_ibrav0_sym/'
fns_out = get_filenames(path,fn_pattern='qe.out',dir_pattern=None)
fns_in = get_filenames(path,fn_pattern='qe.in',dir_pattern=None)

good = []
to_check = []
pbar = tqdm_notebook(total=len(fns_out))
for it,(fn_in,fn_out) in enumerate(zip(fns_in,fns_out)):

    ss = get_patterns(fn_out,str_patterns=['Sym. Ops.','warning: symmetry operation #','No symmetry found',
                                          'additional sym.ops. were found but ignored'])
    Nsym_f = 0
    if ss['additional sym.ops. were found but ignored']:
        s = ss['additional sym.ops. were found but ignored'][0]
        Nsym_f += extract_floats(s)[0]
    if ss['warning: symmetry operation #']:
        nums = []
        for s in ss['warning: symmetry operation #']:
            nums.append(extract_floats(s)[0])

        nums = np.array(nums).max()
        Nsym_f = nums.max()
        
    elif ss['No symmetry found']:
        Nsym_f += 1
    else:

        s = ss['Sym. Ops.'][0]
        
        Nsym_f += extract_floats(s)[0]

    cc = read(fn_in)
    sg = fn2sg(fn_in)
    
    symprec = get_symprec(cc,sg)
    syms = spg.get_symmetry(cc,symprec=symprec)
    Nsym_s = syms['rotations'].shape[0]

    if Nsym_s == Nsym_f:
        good.append(fn_out)
    else:
        to_check.append([it,fn_out,sg,Nsym_s,Nsym_f])

    pbar.update()
pbar.close()

In [274]:
len(to_check)

49

In [277]:
to_check

[[8, '../test_run/check_ibrav0_sym/sg_73-f_8-0/qe.out', 73, 8, 4.0],
 [16, '../test_run/check_ibrav0_sym/sg_211-f_3-0/qe.out', 211, 24, 48.0],
 [17, '../test_run/check_ibrav0_sym/sg_204-f_1-0/qe.out', 204, 24, 48.0],
 [19, '../test_run/check_ibrav0_sym/sg_36-f_7-0/qe.out', 36, 8, 4.0],
 [26, '../test_run/check_ibrav0_sym/sg_15-f_8-0/qe.out', 15, 4, 2.0],
 [27, '../test_run/check_ibrav0_sym/sg_64-f_3-0/qe.out', 64, 8, 4.0],
 [30, '../test_run/check_ibrav0_sym/sg_121-f_4-0/qe.out', 121, 8, 16.0],
 [34, '../test_run/check_ibrav0_sym/sg_204-f_0-0/qe.out', 204, 24, 48.0],
 [38, '../test_run/check_ibrav0_sym/sg_64-f_1-0/qe.out', 64, 8, 4.0],
 [43, '../test_run/check_ibrav0_sym/sg_88-f_1-0/qe.out', 88, 8, 1],
 [45, '../test_run/check_ibrav0_sym/sg_74-f_2-0/qe.out', 74, 8, 4.0],
 [50, '../test_run/check_ibrav0_sym/sg_68-f_5-0/qe.out', 68, 8, 4.0],
 [57, '../test_run/check_ibrav0_sym/sg_119-f_0-0/qe.out', 119, 8, 16.0],
 [67, '../test_run/check_ibrav0_sym/sg_15-f_5-0/qe.out', 15, 4, 2.0],
 [83,

In [275]:
print to_check[4]

[26, '../test_run/check_ibrav0_sym/sg_15-f_8-0/qe.out', 15, 4, 2.0]


In [282]:
it = 43
fn = fns_out[it]
print fn
ss = get_patterns(fn,str_patterns=['Sym. Ops.','warning: symmetry operation #','No symmetry found'])

if ss['warning: symmetry operation #']:
    nums = []
    for s in ss['warning: symmetry operation #']:
        nums.append(extract_floats(s)[0])
    
    nums = np.array(nums).max()
    Nsym_f = nums.max()
elif ss['No symmetry found']:
    Nsym_f += 1
else:
#     print ss['Sym. Ops.']
    s = ss['Sym. Ops.'][0]
    Nsym_f = extract_floats(s)[0]
    
fn = fns_in[it]
cc = read(fn)
sg = fn2sg(fn)
if sg == 36:
    Nsym_s = 4
else:
    symprec = get_symprec(cc,sg)
    syms = spg.get_symmetry(cc,symprec=symprec)
    Nsym_s = syms['rotations'].shape[0]

print Nsym_s == Nsym_f
print s
print Nsym_s

../test_run/check_ibrav0_sym/sg_88-f_1-0/qe.out
False
      8 Sym. Ops. (no inversion) found ( 4 have fractional translation)
8


In [268]:
view(cc)

In [254]:
sg,it = 211,3
cc = crystals[sg][it]
dd,_,_ = get_ibrav0_frame(cc,sg)
view(dd)
view(cc)
print dd.get_cell()

[[-5.52491889  5.52491889  5.52491889]
 [ 5.52491889 -5.52491889  5.52491889]
 [ 5.52491889  5.52491889 -5.52491889]]


In [279]:
missClassificationCorrection[121]

KeyError: 121

In [256]:

symprec = get_symprec(dd,sg)
syms = spg.get_symmetry(dd,symprec=symprec)
Nsym_s = syms['rotations'].shape[0]
print Nsym_s

24


In [261]:
it = 16
fn = fns_in[it]
cc = read(fn)
sg = fn2sg(fn)
symprec = get_symprec(cc,sg)
syms = spg.get_symmetry(cc,symprec=symprec)
Nsym_s = syms['rotations'].shape[0]
print Nsym_s

24


In [262]:
view(cc)